## Waze for Cities Response Processing

El proceso a continuación descarga datos recopilados por la herramienta "Traffic View" de Waze for Cities sobre los cinco tramos de la Avenida Alfonso Reyes que se observan al momento.

In [1]:
# Importamos librerías

from urllib.request import urlopen
import json
import pandas as pd

Los datos que recopila "Traffic View" son guardados en un archivo tipo json en formato de URL. Este URL es permanente, lo cual nos permite estar descargando los datos de una misma liga siempre. A continuación se muestra cómo:

In [2]:
# Guardamos el URL del archivo tipo json para trabajar con él como un diccionario

url = "https://www.waze.com/row-rtserver/broadcast/BroadcastRSS?format=JSON&buid=1397c15e3dfa4f4f7d815e17dd893f4d"
response = urlopen(url)
data = json.loads(response.read())

El archivo que cargamos a la variable `data` es esencialmente un diccionario. Dentro de esta variable existen diversas llaves de las cuales tan solo una (`routes`) es relevante para obtener y trabajar con los datos de tiempo de traslado de los tramos observados en la Av. Alfonso Reyes. La celda a continuación toma esa llave y sus datos correspondientes y los guarda en un DataFrame.

In [3]:
# Guardamos la llave 'routes' en un DataFrame

routes = pd.DataFrame.from_dict(data['routes'])

# Removemos las columnas que no nos interesan
routes = routes.drop(['toName', 'line', 'bbox', 'fromName',
                      'jamLevel', 'id', 'type', 'jams'],
                     axis = 1)

El siguiente paso es crear nuestra base de datos con todos los datos relevantes limpios y listos para ser procesados. El primer paso es crear un DataFrame vacío e ir agregando las columnas relevantes.

In [4]:
## Creamos un dummy DataFrame para crear nuestra base de datos

travel_time_ar = pd.DataFrame()

# Creamos una columna con los nombres de cada ruta
travel_time_ar['ruta'] = routes['name']

# Creamos una columna con la distancia de cada ruta (en metros)
travel_time_ar['distancia'] = routes['length']

# Creamos una columna con el tiempo de traslado (en segundos)
travel_time_ar['travel_time'] = routes['time']

# Creamos una columna con el tiempo histórico de traslado (en segundos)
travel_time_ar['historic_time'] = routes['historicTime']

## Trabajo con Fechas

# A nuestra base de datos le añadimos 
# una columna con el timestamp actual
travel_time_ar['fecha'] = pd.Timestamp.now()

# Creaamos una columna que redondea la hora
# del timestamp al minuto más cercano
travel_time_ar['hora'] = travel_time_ar['fecha'].dt.floor('T')

# A la columna 'hora' le dejamos 
# tan solo la hora del timestamp
travel_time_ar['hora'] = travel_time_ar['hora'].dt.time

# A la columna 'fecha' le dejamos
# tan solo la fecha actual
travel_time_ar['fecha'] = travel_time_ar['fecha'].dt.date

Finalmente, reordenamos las columnas de la base de datos y realizamos una transformación importante: convertimos la unidad de distancia de metros a kilómetros.

In [5]:
# Reordenamos las columnas de nuestro DataFrame
# para comenzar con la fecha y hora del reporte
reorder = ['fecha', 'hora', 'ruta',
           'distancia', 'travel_time', 'historic_time']

travel_time_ar = travel_time_ar.reindex(columns = reorder)

# Transformamos la distancia de las rutas a kilómetros
travel_time_ar['distancia'] = (travel_time_ar['distancia'] / 1000).round(decimals = 1)

Es posible transformar la unidad de tiempo de traslado de segundos un formato de MM:SS, sin embargo, este formato no es agradable para hacer trabajos con gráficas o para realizar operaciones. Si se desea trabajar con un formato de ese estilo, las líneas abajo cumplen esa función:

`travel_time_ar['travel_time'] = travel_time_ar['travel_time'].astype('datetime64[s]').dt.strftime("%M:%S")`
`travel_time_ar['historic_time'] = travel_time_ar['historic_time'].astype('datetime64[s]').dt.strftime("%M:%S")`

Por ahora, la base de datos deja la unidad de tiempo de traslado en el número de segundos. # El resultado es un DataFrame con 6 columnas y 10 filas donde cada fila presenta la fecha y hora de la observación, al igual que el nombre de la ruta, su distancia y sus tiempos de traslado.

In [6]:
travel_time_ar

,fecha,hora,ruta,distancia,travel_time,historic_time
0,2021-09-03,11:14:00,Las Olimpiadas - Gómez Morín,2.6,254,228
1,2021-09-03,11:14:00,Gómez Morín - Las Olimpiadas,2.6,243,248
2,2021-09-03,11:14:00,UDEM - Gómez Morín,6.6,585,565
3,2021-09-03,11:14:00,Gómez Morín - UDEM,6.7,600,565
4,2021-09-03,11:14:00,UDEM - San Agustín,9.5,862,821
5,2021-09-03,11:14:00,San Agustín - UDEM,9.5,915,854
6,2021-09-03,11:14:00,UDEM - Pedro Ramírez V,11.9,1151,1120
7,2021-09-03,11:14:00,Pedro Ramírez V - UDEM,11.8,1148,1066
8,2021-09-03,11:14:00,UDEM - Las Olimpiadas,4.0,328,338
9,2021-09-03,11:14:00,Las Olimpiadas - UDEM,4.0,351,311


In [7]:
# Agregamos los datos nuevos a la base de datos

# Cargamos el archivo de la base
base_completa = pd.read_excel(r"travel_time_ar.xlsx")

# Python lee la columna de fecha como timestamp
# La transformamos para que sea tan solo la fecha
base_completa['fecha'] = base_completa['fecha'].dt.date

# Concatenamos los datos nuevos a la base completa
base_completa = pd.concat([base_completa, travel_time_ar])

# Guardamos el archivo de excel actualizado de la base completa
writer_completa = pd.ExcelWriter(r"travel_time_ar.xlsx",
                                 engine = "xlsxwriter")

base_completa.to_excel(writer_completa, index = False)

writer_completa.save()